<a href="https://colab.research.google.com/github/Arifuzzamanjoy/Bone-Conducted-Speech-Enhancement-With-Neural-Network/blob/main/Objective_test_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa
!pip install pesq
!pip install pystoi
!pip install natsort
import librosa
import numpy as np
from pystoi import stoi
import numpy as np
from scipy.signal import stft

import os
from natsort import natsorted
from pesq import pesq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 31.2 MB/s eta 0:00:00
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23706 sha256=b99270531f824a78fd459f1bddd80861f3cde3a0ec61d52d26b96b0298fe63ec
  Stored in directory: /root/.cache/pip/wheels/da/4b/39/c5f6c4ee93b43281dda4dab5ac5f2bdf9d11074d427493cd55
Successfully built audioread

[notice] A new release of

LSD

In [ ]:
#####################NEW#######

import librosa
import numpy as np
import os

# Define the paths of the subfolders
clean_folder = "/work/DNN/clean"
noise_folder = "/work/DNN/noisy"
enhance_folder = "/work/DNN/enhance"

# Initialize an empty list
# to store the LSD values
lsd_c_n=np.array([])
lsd_c_e=np.array([])
stoi_c_e=np.array([])
stoi_c_n=np.array([])
pesq_c_n=np.array([])
pesq_c_e=np.array([])


# read all audio from clean_folder and noise_folder
clean_files = os.listdir(clean_folder)
noise_files = os.listdir(noise_folder)
enhance_files=os.listdir(enhance_folder)

# sort the file names using natural sorting
clean_files = natsorted(clean_files)
noise_files = natsorted(noise_files)
sr=16000

# Loop through two audio file in clean_files and noise_files those flow natural sorting as an windows
#follow natural sorting while looping through the files in the folder
for clean_file,noise_file,enhance_file in zip(natsorted(clean_files),natsorted(noise_files),natsorted(enhance_files)):
    # Read the clean audio file
    clean_audio,sr=librosa.load(os.path.join(clean_folder,clean_file),sr=sr)
    # Read the noise audio file
    noise_audio,sr=librosa.load(os.path.join(noise_folder,noise_file),sr=sr)
    # Read the enhance audio file
    enhance_audio,sr=librosa.load(os.path.join(enhance_folder,enhance_file),sr=sr)
    # Calculate the LSD value
    if len(clean_audio) > len(noise_audio):
        clean_audio = clean_audio[:len(noise_audio)]
    else:
        noise_audio = noise_audio[:len(clean_audio)]


    # Compute STFT of clean and noisy audio
    f, t, Zxx_noisy = stft(noise_audio, sr)
    f, t, Zxx_clean = stft(clean_audio, sr)
    #take log of the magnitude of the STFT
    log_Zxx_noisy = np.log(np.abs(Zxx_noisy))
    log_Zxx_clean = np.log(np.abs(Zxx_clean))
    # Compute mean squared error between log spectra
    mse = np.mean((log_Zxx_noisy - log_Zxx_clean) ** 2)

# Take square root to get LSD
    lsd = np.sqrt(mse)
    lsd_c_n = np.append(lsd_c_n, lsd)

#PSEQ
    # pesq between clean_audio,noise_audio
    pesq_socre_yc_yn = pesq(sr,clean_audio,noise_audio,'nb')
    pesq_c_n=np.append(pesq_c_n,pesq_socre_yc_yn)


#STOI
    stoi_value=stoi(clean_audio,noise_audio,sr,extended=False)
    # Append the stoi_value to the list
    stoi_c_n=np.append(stoi_c_n,stoi_value)




# Stake LSD

    if len(clean_audio) > len(enhance_audio):
        clean_audio = clean_audio[:len(enhance_audio)]
    else:
        enhance_audio = enhance_audio[:len(clean_audio)]

# Compute STFT of enhance and clean audio
    f, t, Zxx_enhance = stft(enhance_audio, sr)
    f, t, Zxx_clean = stft(clean_audio, sr)
#take log of the magnitude of the STFT
    log_Zxx_enhance = np.log(np.abs(Zxx_enhance))
    log_Zxx_clean = np.log(np.abs(Zxx_clean))
# Compute mean squared error between log spectra
    mseenhance = np.mean((log_Zxx_enhance - log_Zxx_clean) ** 2)
    lsdenhance = np.sqrt(mseenhance)
    lsd_c_e = np.append(lsd_c_e, lsdenhance)



#peseq

    pesq_socre_yc_ye = pesq(sr,clean_audio,enhance_audio,'nb')
    # pesq between clean_audio,enhance_audio
    pesq_c_e=np.append(pesq_c_e,pesq_socre_yc_ye)

#STOI
    stoi_value1=stoi(clean_audio,enhance_audio,sr,extended=False)
    # Append the stoi_value to the list
    stoi_c_e=np.append(stoi_c_e,stoi_value1)



In [ ]:
import pandas as pd
df = pd.DataFrame(clean_files, columns = ['File name']) #stoi_Noise_values   stoi_values
df['LSD C-N'] = lsd_c_n
df['LSD C-E']=lsd_c_e
df['LSD Distance Improvement']=df['LSD C-N']-df['LSD C-E']


df["stoi-c-e"]=stoi_c_e
df["stoi-c-n"]=stoi_c_n
df["stoi improvement"]=stoi_c_e-stoi_c_n

df["pesq-c-n"]=pesq_c_n
df["pesq-c-e"]=pesq_c_e
df["pesq_improvement"]=pesq_c_n-pesq_c_e

In [ ]:

female_mask = df['File name'].str.startswith('F') # True if file name starts with F
male_mask = df['File name'].str.startswith('M') # True if file name starts with M
df['gender'] = np.where(female_mask, 'Female', np.where(male_mask, 'Male', 'unknown')) # Assign values based on masks
df.head(10) # Show the first 5 rows of the DataFrame


,File name,LSD C-N,LSD C-E,LSD Distance Improvement,stoi-c-e,stoi-c-n,stoi improvement,pesq-c-n,pesq-c-e,pesq_improvement,gender
0,F001_A16_ch1_1_1.wav,1.749043,1.652387,0.096656,0.841822,0.661494,0.180328,1.831002,1.806397,0.024605,Female
1,F001_A17_ch1_1_1.wav,1.741740,1.621497,0.120243,0.818812,0.673117,0.145695,1.637576,1.906026,-0.268450,Female
2,F002_A01_ch1_1_1.wav,1.811491,1.363109,0.448382,0.905197,0.694511,0.210686,1.456603,2.124494,-0.667891,Female
3,F002_A07_ch1_1_1.wav,1.692503,1.334429,0.358074,0.885510,0.705402,0.180108,1.551466,2.145295,-0.593828,Female
4,F002_A10_ch1_1_1.wav,1.869078,1.388567,0.480511,0.902404,0.666274,0.236130,1.427620,2.313601,-0.885981,Female
5,F002_A15_ch1_1_1.wav,1.778791,1.385267,0.393525,0.899714,0.730058,0.169656,1.605118,2.188458,-0.583341,Female
6,F003_A01_ch1_1_1.wav,1.845749,1.312400,0.533350,0.795937,0.664170,0.131766,1.762093,2.318969,-0.556876,Female
7,F003_A17_ch1_1_1.wav,1.853648,1.284755,0.568892,0.803276,0.646000,0.157277,1.787513,2.448894,-0.661381,Female
8,F003_A19_ch1_1_1.wav,1.835517,1.350710,0.484807,0.810025,0.657535,0.152491,1.771597,2.415909,-0.644311,Female
9,M001_A05_ch1_1_1.wav,1.801254,1.446115,0.355138,0.899059,0.722245,0.176814,1.561776,2.317502,-0.755726,Male


In [ ]:
df.to_csv('data1.csv', index=False)

In [ ]:
DeepnoteChart(df, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}}}""")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8932d5f0-83a3-437e-bb6a-bbc3eadd6e87' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>